In [1]:
def RP_TargetVol(rets, Target, lb, ub) :

    covmat = DataFrame.cov(rets)
    
    def annualize_scale(rets):
    
        med  = np.median(np.diff(rets.index.values))
        seconds = int(med.astype('timedelta64[s]').item().total_seconds())
        if seconds < 60:
            freq = 'second'.format(seconds)
        elif seconds < 3600:
            freq = 'minute'.format(seconds//60)
        elif seconds < 86400:
            freq = 'hour'.format(seconds//3600)
        elif seconds < 604800:
            freq = 'day'.format(seconds//86400)
        elif seconds < 2678400:
            freq = 'week'.format(seconds//604800)
        elif seconds < 7948800:
            freq = 'month'.format(seconds//2678400)
        else:
            freq = 'quarter'.format(seconds//7948800)
        
        def switch1(x): 
            return {
            'day' : 252,
            'week' : 52,
            'month' : 12,
            'quarter' : 4,
            }.get(x)
        
        return switch1(freq)


    #--- Risk Budget Portfolio Objective Function ---#

    def RiskParity_objective(x) :
            
            variance = x.T @ covmat @ x
            sigma = variance ** 0.5
            mrc = 1/sigma * (covmat @ x)
            rc = x * mrc
            a = np.reshape(rc, (len(rc), 1))
            risk_diffs = a - a.T
            sum_risk_diffs_squared = np.sum(np.square(np.ravel(risk_diffs)))
            return (sum_risk_diffs_squared)
            
    
    #--- Constraints ---#
     
    def TargetVol_const_lower(x) :
        
        variance = x.T @ covmat @ x
        sigma = variance ** 0.5 
        sigma_scale = sigma * np.sqrt(annualize_scale(rets)) 
        
        vol_diffs = sigma_scale - (Target * 0.95)
        return(vol_diffs)  

    def TargetVol_const_upper(x) :
        
        variance = x.T @ covmat @ x
        sigma = variance ** 0.5 
        sigma_scale = sigma * np.sqrt(annualize_scale(rets)) 
        
        vol_diffs = (Target * 1.05) - sigma_scale
        return(vol_diffs)      
    
    
    #--- Calculate Portfolio ---#
    
    x0 = np.repeat(1/covmat.shape[1], covmat.shape[1]) 
    lbound  = np.repeat(lb, covmat.shape[1])
    ubound  = np.repeat(ub, covmat.shape[1])
    bnds = tuple(zip(lbound, ubound))
    constraints = ({'type': 'ineq', 'fun': TargetVol_const_lower},
                   {'type': 'ineq', 'fun': TargetVol_const_upper})
    options = {'ftol': 1e-20, 'maxiter': 5000}
    
    result = minimize(fun = RiskParity_objective,
                      x0 = x0,
                      method = 'SLSQP',
                      constraints = constraints,
                      options = options,
                      bounds = bnds)
    return(result.x)

In [2]:
def annualize_scale(rets):
    
        med  = np.median(np.diff(rets.index.values))
        seconds = int(med.astype('timedelta64[s]').item().total_seconds())
        if seconds < 60:
            freq = 'second'.format(seconds)
        elif seconds < 3600:
            freq = 'minute'.format(seconds//60)
        elif seconds < 86400:
            freq = 'hour'.format(seconds//3600)
        elif seconds < 604800:
            freq = 'day'.format(seconds//86400)
        elif seconds < 2678400:
            freq = 'week'.format(seconds//604800)
        elif seconds < 7948800:
            freq = 'month'.format(seconds//2678400)
        else:
            freq = 'quarter'.format(seconds//7948800)
        
        def switch1(x): 
            return {
            'day' : 252,
            'week' : 52,
            'month' : 12,
            'quarter' : 4,
            }.get(x)
        
        return switch1(freq)

In [3]:
def RiskParity_objective(x) :
            
            variance = x.T @ covmat @ x
            sigma = variance ** 0.5
            mrc = 1/sigma * (covmat @ x)
            rc = x * mrc
            a = np.reshape(rc, (len(rc), 1))
            risk_diffs = a - a.T
            sum_risk_diffs_squared = np.sum(np.square(np.ravel(risk_diffs)))
            return (sum_risk_diffs_squared)

In [5]:
def TargetVol_const_lower(x) :
        
        variance = x.T @ covmat @ x
        sigma = variance ** 0.5 
        sigma_scale = sigma * np.sqrt(annualize_scale(rets)) 
        
        vol_diffs = sigma_scale - (Target * 0.95)
        return(vol_diffs)  

    def TargetVol_const_upper(x) :
        
        variance = x.T @ covmat @ x
        sigma = variance ** 0.5 
        sigma_scale = sigma * np.sqrt(annualize_scale(rets)) 
        
        vol_diffs = (Target * 1.05) - sigma_scale
        return(vol_diffs)
    
    x0 = np.repeat(1/covmat.shape[1], covmat.shape[1]) 
    lbound  = np.repeat(lb, covmat.shape[1])
    ubound  = np.repeat(ub, covmat.shape[1])
    bnds = tuple(zip(lbound, ubound))
    constraints = ({'type': 'ineq', 'fun': TargetVol_const_lower},
                   {'type': 'ineq', 'fun': TargetVol_const_upper})
    options = {'ftol': 1e-20, 'maxiter': 5000}
    
    result = minimize(fun = RiskParity_objective,
                      x0 = x0,
                      method = 'SLSQP',
                      constraints = constraints,
                      options = options,
                      bounds = bnds)
    return(result.x)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 10)

In [6]:
wts_cash = DataFrame(1 - wts_tv.sum(axis = 1))
wts_cash.columns = ['Cash']
wts_new = pd.concat([wts_tv, wts_cash], axis = 1)

rets_cash = pd.DataFrame(data = np.repeat(0.00, rets.shape[0]).reshape(rets.shape[0], 1),
                         index = rets.index,
                         columns = ['Cash'])
rets_new = pd.concat([rets, rets_cash], axis = 1)

NameError: name 'DataFrame' is not defined